# Implémentation d'un _Markov Chain Monte Carlo_ (MCMC) avec l'algorithme de Metropolis.

Nous allons suivre les étapes décrite dans l'excellent article d'introduction au MCMC de David W. Hogg et Daniel Foreman-Mackey, disponible [ici](https://ui.adsabs.harvard.edu/abs/2018ApJS..236...11H/abstract).
La section 3 sera particulièrement utile pour cet exemple.

Vous pouvez installer `tqdm` si vous souhaitez afficher un indicateur de progrès dans la boucle de MCMC.
Pour l'installation, la commande `python -m pip install tqdm` devrait fonctionner.
Pour l'utilisation, il suffit de placer votre générateur (`range()`) dans la fonction `tqdm.tqdm()`:

```python
for i in tqdm.tqdm(range(10)):
    # do something
    pass
```

In [ ]:
from collections.abc import Callable
import numpy as np
import matplotlib.pyplot as plt
import tqdm

rng = np.random.default_rng()

## Fonction de densité unidimensionnelle (Problèmes 2 et 3 de l'article)
### Densité Gaussienne

Pour ce premier exercice, nous allons implémenter l'algorithme de Metropolis et l'appliquer à une distribution normale unidimensionnelle.

Utilisez les informations suivantes:

- La fonction de densité $p(\theta)$ est une gaussienne à une dimension avec moyenne de $\mu=2$ et une variance $\sigma^2=2$.
- La distribution de proposition $q(\theta'|\theta)$ est une gaussienne pour $\theta'$ avec une moyenne $\mu=\theta$ et un écart type $\sigma = 1$.
- Le point initial du MCMC est $\theta = 0$.
- Le MCMC doit performer $10^4$ itérations.

L'équation de la distribution normale est

$$
p(\theta) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp\left[ -\frac{(\theta - \mu)^2}{2 \sigma^2}\right].
$$

Or, pour éviter les erreurs numériques, on utilise son logarithme. **Codez directement une fonction pour $\ln p(\theta)$** (n'utilisez pas simplement `np.log` sur une gaussienne).

In [ ]:
def log_gaussian(x, mean=2, var=2) -> float:
    # TODO: ln d'une distribution normale 1D
    pass

On peut maintenant implémenter l'algorithme de Metropolis.

On souhaite que notre algorithme soit applicable à n'importe quelle densité de (log-)probabilité qui accepte un argument $\theta$ scalaire. On peut donc donner `log_density` (notre fonction de probabilité ci-dessus) en argument à la fonction.

In [ ]:
def mcmc_metropolis(
    log_density: Callable,
    theta0: float,
    nsteps: int,
    q_scale: float = 1.0
) -> np.ndarray[float]:
    """
    - log_density: fonction de log-densité, accepte un argument theta
    - theta0: valeur initiale de theta pour le MCMC
    - nsteps: nombre de pas à faire dans le MCMC
    - q_scale: écart type de la distribution de proposition.

    La fonction retourne un tableau d'échantillons pour theta.
    """
    pass

Appliquez l'algorithme pour obtenir 10000 échantillons.

Affichez ensuite un histogramme et comparez le avec la PDF analytique.
Affichez ensuite l'évolution temporelle ($\theta$ vs $k$) du MCMC.

### Impact de l'échelle de la distribution de proposition

Testez différentes échelles pour la distribution de proposition (`q_scale`). Quel est l'impact sur l'échantillonnage? Testez `q_scale=10000` et `q_scale=1e-5`.

### Distribution Uniforme

Réutilisez votre MCMC pour échantillonner une distribution uniforme entre 3 et 7, soit $\mathcal{U}(0, 7)$. Vous devrez encore une fois coder le log de cette densité de probabilité. Tout le reste dans votre `mcmc_metropolis` devrait fonctionner.

Attention à la valeur d'initialisation!

## Fonction de densité 2D

Pour échantilloner un problème plus complexe, on peut généraliser le code ci-dessus à une distribution 2D.

### Densité de probabilité

Nous allons échantillonner une distribution $p(\theta)$ où $\theta$ contient deux dimensions. La distribution sera une distribution normale bidimensionnell avec une une moyenne

$$
\mu = \begin{bmatrix}0\\ 0\end{bmatrix}
$$

et une matrice de covariance:

$$
V = \begin{bmatrix}2.0 & 1.2\\1.2 & 2.0\end{bmatrix}.
$$

L'équation d'une normale à N dimensions est donnée par:

$$
p(\theta) = (2\pi)^{-N/2} \det{(V)}^{-1/2} \exp{\left(-\frac{1}{2}(\theta - \mu)^T V^{-1} (\theta - \mu)\right)}
$$

Vous pouvez coder le log de cette densité dans la cellule ci-dessous. Elle devrait accepter un paramètre `theta` contenant deux valeurs.

In [ ]:
def log_gaussian_density_2d(x: np.ndarray[float]) -> float:
    x = np.asarray(x)
    ndim = 2
    mu = np.array([0.0, 0.0])
    cov = np.array([[2.0, 1.2], [1.2, 2.0]])
    assert len(x) == ndim, (
        f"Wrong number of input dimensions. Got {len(x)}, expected {ndim}"
    )
    # @ est une multiplication matricielle. Équivalent à np.dot ou np.matmul.
    p = (
        -0.5 * np.log(np.linalg.det(cov)) - 0.5 * ndim * (2 * np.pi)
        - 0.5 * (x - mu) @ np.linalg.inv(cov) @ (x - mu)
    )
    return p

Comme on connait la probabilité analytique et qu'elle se calcule dans un temps raisonnable, on peut l'afficher sur une grille pour s'assurer que la fonction le résultat attendu.
Ce sera aussi utile pour vérifier que nos échantillons MCMC donnent la bonne distribution.

### Algorithme de Metropolis

Pour l'algorithme de Metropolis, vous pouvez copier votre fonction précédente et la modifier pour qu'elle fonctionne avec un paramètre $\theta$ 2D.

Utilisez une distribution de proposition normale 2D avec une matrice de covariance identité. `rng.multivariate_normal` devrait fonctionner.

Affichez ensuite l'histogramme pour chaque paramètre, puis l'évolution temporelle de chaque paramètre.

Affichez également un nuage de point 2D ou un histogramme 2D.

In [ ]:
from typing import Union

def mcmc_metropolis(
    log_density: Callable,
    theta0: np.ndarray[float],
    nsteps: int,
    q_var: Union[float, np.ndarray[float]] = 1.0
) -> np.ndarray[float]:
    """
    - log_density: fonction de log-densité, accepte un argument theta
    - theta0: valeur initiale de theta pour le MCMC
    - nsteps: nombre de pas à faire dans le MCMC
    - q_var: variance ou covariance de la distribution de proposition. Peut-être:
      - Un scalaire si tous les paramètres ont la même variance et que la covariance est 0.
      - Un vecteur de la même taille que theta0 si la covariance est 0 mais que les variances sont différentes.
      - Une matrice de covariance ndim x ndim.

    La fonction retourne un tableau d'échantillons pour chaque paramètre
    avec un format (nsteps, ndim).
    """
    pass

Pour afficher les histogrammes à plusieurs dimensions, un _corner plot_ est souvent utile. Il s'agit d'un graphique montrant la distribution 2D ainsi que les distributions marginales.

Pour l'utiliser, il faudra installer `corner` (`python -m pip install corner`).

Voici un lien vers la documentation: https://corner.readthedocs.io/en/latest/

## Bonus: MCMC appliqué à l'analyse de données

Utilisez votre MCMC ci-dessus pour échantillonner la distribution à posteriori du devoir 1.
Vous devrez copier votre code pour le modèle, les distributions et l'importation des données.

Dans ce cas-ci, vous pouvez définir une distribution `log_posterior` en additionnant le log du prior et de la vraisemblance avec `np.log()`. Quelque chose comme:

```python
def log_posterior(param):
    lp = np.log(prior(param))
    if not np.isfinite(lp):
        return - np.inf
    return lp + np.log(likelihood(param, x, data, edata))
```

_Note: En général, il est préférable de coder toutes les distributions en log directement. Nous utilisons `np.log()` ici pour pouvoir réutiliser les fonctions du devoir 1, et car nous savons qu'il n'y aura pas de problèmes numériques dans ce cas spécifique_